In [1]:
import os
import sys
src_path = os.path.split(os.getcwd())[0]
sys.path.insert(0, src_path)

import json
import random
import glob

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.datasets as datasets
import torch.nn as nn
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, balanced_accuracy_score

from PIL import Image

from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, RandomResizedCrop, InterpolationMode
import matplotlib.pyplot as plt

import clip_.clip as clip
from clip_.clip import _transform
from clip_.model import CLIPGeneral

import shutil

from torch.utils.data import Dataset
import torch

checkpoint_paths = [
    '/data/checkpoints/checkpoints_icml22/yfcc/clip_rn50_yfcc_epoch_28.pt',
    '/data/checkpoints/checkpoints_icml22/yfcc/cloob_rn50_yfcc_epoch_28.pt',
]

device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [2]:
embedding_target_df = pd.read_csv('/system/user/publicdata/MIMIC_CXR/hageneder/frontal_with_embedding.csv')
embedding_target_df = embedding_target_df[['study_id', 'jpg_path', 'embedding_path_cloob_rn50_yfcc_epoch_28', 'embedding_path_clip_rn50_yfcc_epoch_28', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']]
embedding_target_df

,study_id,jpg_path,embedding_path_cloob_rn50_yfcc_epoch_28,embedding_path_clip_rn50_yfcc_epoch_28,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,57682201,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,1,0,0,0,0,0,0,0,0,0,0,0
1,55032644,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,0,0,0
2,54345212,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,1,0,1,0,0,0,0,0,0,0,0,1
3,50963033,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,1,1,0,1,0,0,0,1,0,0,0,0,1
4,58958645,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242418,55314230,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,1,0,0,0,0,0,1,1,0,0,0,1
242419,55314230,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,1,0,0,0,0,0,1,1,0,0,0,1
242420,53699756,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,1,0,0
242421,59644954,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# Assuming embedding_target_df is your DataFrame

# Selecting the target columns
target_columns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
                  'Lung Lesion', 'Lung Opacity', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
                  'Support Devices']

# Define the number of samples per target column
n_samples_per_column = 2500

# Initialize an empty DataFrame to store the stratified samples
stratified_samples = pd.DataFrame()

# Perform stratified sampling on each target column
for column in target_columns:
    stratified_split = StratifiedShuffleSplit(n_splits=1, train_size=n_samples_per_column, random_state=42)
    _, selected_indices = next(stratified_split.split(embedding_target_df, embedding_target_df[column]))
    column_samples = embedding_target_df.iloc[selected_indices]
    stratified_samples = pd.concat([stratified_samples, column_samples])

# Limit the number of rows in stratified_samples to be equal to n_samples_per_column * len(target_columns)
stratified_samples = stratified_samples.sample(n=n_samples_per_column * len(target_columns), random_state=42)

In [4]:
stratified_samples.reset_index(drop=True, inplace=True)
stratified_samples

,study_id,jpg_path,embedding_path_cloob_rn50_yfcc_epoch_28,embedding_path_clip_rn50_yfcc_epoch_28,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,59848517,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,1,0,0,0,0,1
1,54973925,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,0,0,0
2,50444919,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,1,1,0,0,0,0
3,56438603,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,1,0,0,0,0,0,0,0,0,0
4,54322203,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,1,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32495,52193692,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,0,0,0
32496,53099224,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,0,0,0,0,0
32497,53129082,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,1,0,1,0,0,0,0,1,0,0,0,0
32498,50353404,/system/user/publicdata/MIMIC_CXR/hageneder/JP...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,/system/user/publicdata/MIMIC_CXR/hageneder/em...,0,0,0,0,0,0,0,0,1,0,0,0,0


In [5]:
def load_checkpoint(checkpoint_path):
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path)
    model_config_file = os.path.join('/data/model_config', checkpoint['model_config_file'])
    
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print(" ")
    print("Device is", device)

    # Load model config
    assert os.path.exists(model_config_file)
    with open(model_config_file, 'r') as f:
        model_info = json.load(f)
    model = CLIPGeneral(**model_info)
    preprocess = _transform(model.visual.input_resolution, is_train=False)

    # Load model state dictionary
    sd = checkpoint["state_dict"]
    sd = {k[len('module.'):]: v for k, v in sd.items()}  # Remove 'module.' prefix from keys for DataParallel
    if 'logit_scale_hopfield' in sd:
        sd.pop('logit_scale_hopfield', None)
    model.load_state_dict(sd)

    model = model.to(device)
    model.eval()

    return model, preprocess, device


def zero_shot_classifier(model, classnames, templates, device):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in classnames:
            texts = [template(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).to(device) #tokenize
            class_embeddings = model.encode_text(texts)  # Access the encode_text method from model.module
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).to(device)
    return zeroshot_weights

def run(model, classifier, dataloader, accuracy_metric):
    with torch.no_grad():
        all_logits = []
        all_targets = []
        for embeddings, target in dataloader:
            embeddings = embeddings.to(device)
            target = target.to(device)

            # predict
            embeddings /= embeddings.norm(dim=-1, keepdim=True)
            logits = embeddings @ classifier
            all_logits.append(logits.cpu())
            all_targets.append(target.cpu())

        all_logits = torch.cat(all_logits).squeeze().numpy()
        all_targets = torch.cat(all_targets).numpy()
        
        acc = accuracy_metric(all_targets, all_logits.argmax(axis=1)) * 100.0
        return acc , all_logits.argmax(axis=1) , all_targets

class CustomImageDataset(Dataset):
    def __init__(self, dataframe, embedding_path, target_, transform=None):
        self.dataframe = dataframe
        self.embedding_path = embedding_path
        self.transform = transform
        self.target_ = target_

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        embedding_file = row[self.embedding_path]
        embedding = torch.load(embedding_file, map_location=torch.device("cuda:1"))

        if self.transform is not None:
            embedding = self.transform(embedding) 
        target = self.target_
        label = row[target]
        return embedding, label

    def __len__(self):
        return len(self.dataframe)

In [6]:
from typing import List, Tuple, Optional

classes_CRX_one = ['yes', 'no']

classes_CRX_more = ['No Finding', 'one finding', 'two findings', 'three findings', 'four findings', 'five findings', 'six findings', 'seven findings', 'eight findings', 'nine findings', 'ten findings', 'eleven findings', 'twelve findings', 'thirteen findings']

cxr_labels_yes_no = [
    ['Atelectasis', 'no Atelectasis'],
    ['Cardiomegaly', 'no Cardiomegaly'],
    ['Consolidation', 'no Consolidation'],
    ['Edema', 'no Edema'],
    ['Enlarged Cardiomediastinum', 'no Enlarged Cardiomediastinum'],
    ['Fracture', 'no Fracture'],
    ['Lung Lesion', 'no Lung Lesion'],
    ['Lung Opacity', 'no Lung Opacity'],
    ['Pleural Effusion', 'no Pleural Effusion'],
    ['Pleural Other', 'no Pleural Other'],
    ['Pneumonia', 'no Pneumonia'],
    ['Pneumothorax', 'no Pneumothorax'],
    ['Support Devices', 'no Support Devices']
]


illness_templates_one = [
    lambda c: f'There was {c}.',
    lambda c: f'There is a {c}.',
]

illness_templates_more = [
    lambda c: f'You have {c}.',
]

In [7]:
classnames = cxr_labels_yes_no
prompt_templates = illness_templates_more

In [8]:
embedding_target_df = stratified_samples

In [9]:
df_accuracy_zero = pd.DataFrame(columns=['model'] + target_columns)

# Zero-Shot Classification of Chest X-Ray Images

In [26]:
for checkpoint_path in tqdm(checkpoint_paths):
    for i,target_ in enumerate(tqdm(target_columns)):
        model, preprocess, device = load_checkpoint(checkpoint_path)
        
        # Determine the corresponding embedding paths based on the model
        if checkpoint_path == '/data/checkpoints/checkpoints_icml22/yfcc/clip_rn50_yfcc_epoch_28.pt':
            embedding_path = 'embedding_path_clip_rn50_yfcc_epoch_28'
            print("Using clip model. Embedding path:", embedding_path)
        else:
            embedding_path = 'embedding_path_cloob_rn50_yfcc_epoch_28'
            print("Using cloob model. Embedding path:", embedding_path)
        dataset = CustomImageDataset(embedding_target_df, embedding_path, target_, transform=None)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=256)

        model.eval()
        accuracy_metric = accuracy_score
        
        # Print classname being evaluated
        print("Evaluating zero-shot accuracy for the class:", classnames[i], flush=True)
        
        classifier = zero_shot_classifier(model, classnames[i], prompt_templates, device)
        print("Evaluating zero-shot accuracy for the image dataset", flush=True)
        accuracy, pre, targets = run(model, classifier, dataloader, accuracy_metric)
        print('Zero-shot accuracy:', accuracy.round(2))

        # Add the accuracy to a pandas dataframe and include the checkpoint path (only the last part of the path after the last /)
        new_row = {'model': checkpoint_path.split('/')[-1], target_ : round(accuracy, 2)}
        df_accuracy_zero = pd.concat([df_accuracy_zero, pd.DataFrame(new_row, index=[0])], ignore_index=True)
        torch.cuda.empty_cache()


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

 
Device is cuda:1
Using clip model. Embedding path: embedding_path_clip_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 70.05
 
Device is cuda:1
Using clip model. Embedding path: embedding_path_clip_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 75.38
 
Device is cuda:1
Using clip model. Embedding path: embedding_path_clip_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 28.21
 
Device is cuda:1
Using clip model. Embedding path: embedding_path_clip_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 12.0
 
Device is cuda:1
Using clip model. Embedding path: embedding_pa

  0%|          | 0/13 [00:00<?, ?it/s]

 
Device is cuda:1
Using cloob model. Embedding path: embedding_path_cloob_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 65.08
 
Device is cuda:1
Using cloob model. Embedding path: embedding_path_cloob_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 25.44
 
Device is cuda:1
Using cloob model. Embedding path: embedding_path_cloob_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 85.49
 
Device is cuda:1
Using cloob model. Embedding path: embedding_path_cloob_rn50_yfcc_epoch_28
Running zero-shot classification for all classes of the dataset
Evaluating zero-shot accuracy for the image dataset
Zero-shot accuracy: 87.9
 
Device is cuda:1
Using cloob model. Embedding path: emb

In [27]:
#change the df_accuracy that there are only two rows: one for the clip model and one for the cloob model 
df_accuracy = df_accuracy.groupby(['model']).mean()
df_accuracy = df_accuracy.reset_index()
df_accuracy

,model,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,clip_rn50_yfcc_epoch_28.pt,70.05,75.38,28.21,12.0,96.62,88.62,4.00,27.69,22.41,1.54,90.41,57.18,28.97
1,cloob_rn50_yfcc_epoch_28.pt,65.08,25.44,85.49,87.9,5.69,98.67,97.49,79.59,38.36,57.08,92.77,10.21,41.49


In [28]:
#in the df_accuracy add a new column with overall accuracy and add the mean of the accuracies of the different classes excluding the first column (the model name)
df_accuracy['overall_accuracy'] = df_accuracy.iloc[:, 1:].mean(axis=1)
df_accuracy

,model,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,overall_accuracy
0,clip_rn50_yfcc_epoch_28.pt,70.05,75.38,28.21,12.0,96.62,88.62,4.00,27.69,22.41,1.54,90.41,57.18,28.97,46.390769
1,cloob_rn50_yfcc_epoch_28.pt,65.08,25.44,85.49,87.9,5.69,98.67,97.49,79.59,38.36,57.08,92.77,10.21,41.49,60.404615


In [ ]:
#safe the df as csv and add the length of the image dataset to a new column

df_accuracy['length'] = len(df_test)
df_accuracy.to_csv('/system/user/publicdata/MIMIC_CXR/hageneder/zero_shot_accuracy_one.csv')
df_accuracy

# one shot eval

In [9]:
def run_threshold(model, classifier, dataloader, accuracy_metric, thresholds):
    with torch.no_grad():
        all_results = []
        for threshold in thresholds:
            print("Evaluating threshold:", threshold)
            all_logits = []
            all_targets = []
            for embeddings, target in dataloader:
                embeddings = embeddings.to(device)
                target = target.to(device)
                # Predict
                embeddings /= embeddings.norm(dim=-1, keepdim=True)
                logits = embeddings @ classifier

                # Apply threshold for classification
                probabilities = torch.softmax(logits, dim=-1)
                max_probabilities, max_indices = torch.max(probabilities, dim=-1)
                predictions = torch.where(max_probabilities >= threshold, max_indices, torch.tensor(0).to(device))

                all_logits.append(predictions.cpu())
                all_targets.append(target.cpu())

            all_logits = torch.cat(all_logits).squeeze().numpy()
            all_targets = torch.cat(all_targets).numpy()

            acc = accuracy_metric(all_targets, all_logits) * 100.0
            all_results.append(acc)
        torch.cuda.empty_cache()
        return all_results, all_logits, all_targets



In [10]:
df_accuracy_one = pd.DataFrame(columns=['model'] + target_columns + ['threshold'])

In [11]:
thresholds = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.999]

for checkpoint_path in tqdm(checkpoint_paths):
    for i, target_ in enumerate(tqdm(target_columns)):
        model, preprocess, device = load_checkpoint(checkpoint_path)
        
        # Determine the corresponding embedding paths based on the model
        if checkpoint_path == '/data/checkpoints/checkpoints_icml22/yfcc/clip_rn50_yfcc_epoch_28.pt':
            embedding_path = 'embedding_path_clip_rn50_yfcc_epoch_28'
            print("Using clip model. Embedding path:", embedding_path)
        elif checkpoint_path == '/data/checkpoints/checkpoints_icml22/yfcc/cloob_rn50_yfcc_epoch_28.pt':
            embedding_path = 'embedding_path_cloob_rn50_yfcc_epoch_28'
            print("Using cloob model. Embedding path:", embedding_path)
        dataset = CustomImageDataset(embedding_target_df, embedding_path, target_, transform=None)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=256)

        model.eval()
        accuracy_metric = accuracy_score
        
        # Print classname being evaluated
        print("Evaluating zero-shot accuracy for the class:", classnames[i], flush=True)
        
        classifier = zero_shot_classifier(model, classnames[i], prompt_templates, device)
        print("Evaluating one-shot threshold accuracy for the image dataset", flush=True)
        accuracies, pre, targets = run_threshold(model, classifier, dataloader, accuracy_metric, thresholds)
        
        rounded_accuracies = [round(acc, 2) for acc in accuracies]
        print('one-shot threshold accuracy:', rounded_accuracies)

        # Add the accuracies to a pandas dataframe and include the checkpoint path (only the last part of the path after the last /)
        for j, threshold in enumerate(thresholds):
            new_row = {'model': checkpoint_path.split('/')[-1], target_: rounded_accuracies[j], 'threshold': threshold}
            df_accuracy_one = pd.concat([df_accuracy_one, pd.DataFrame(new_row, index=[0])], ignore_index=True)
        
        torch.cuda.empty_cache()


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

 
Device is cuda:1
Using clip model. Embedding path: embedding_path_clip_rn50_yfcc_epoch_28
Evaluating zero-shot accuracy for the class: ['Atelectasis', 'no Atelectasis']
Evaluating one-shot threshold accuracy for the image dataset
Evaluating threshold: 0.4
Evaluating threshold: 0.5
Evaluating threshold: 0.6
Evaluating threshold: 0.7
Evaluating threshold: 0.8
Evaluating threshold: 0.9
Evaluating threshold: 0.95
Evaluating threshold: 0.99
Evaluating threshold: 0.999
one-shot threshold accuracy: [69.57, 69.57, 80.03, 80.03, 80.03, 80.03, 80.03, 80.03, 80.03]
 
Device is cuda:1
Using clip model. Embedding path: embedding_path_clip_rn50_yfcc_epoch_28
Evaluating zero-shot accuracy for the class: ['Cardiomegaly', 'no Cardiomegaly']
Evaluating one-shot threshold accuracy for the image dataset
Evaluating threshold: 0.4
Evaluating threshold: 0.5
Evaluating threshold: 0.6
Evaluating threshold: 0.7
Evaluating threshold: 0.8
Evaluating threshold: 0.9
Evaluating threshold: 0.95
Evaluating threshold

  0%|          | 0/13 [00:00<?, ?it/s]

 
Device is cuda:1
Using cloob model. Embedding path: embedding_path_cloob_rn50_yfcc_epoch_28
Evaluating zero-shot accuracy for the class: ['Atelectasis', 'no Atelectasis']
Evaluating one-shot threshold accuracy for the image dataset
Evaluating threshold: 0.4
Evaluating threshold: 0.5
Evaluating threshold: 0.6
Evaluating threshold: 0.7
Evaluating threshold: 0.8
Evaluating threshold: 0.9
Evaluating threshold: 0.95
Evaluating threshold: 0.99
Evaluating threshold: 0.999
one-shot threshold accuracy: [65.77, 65.77, 80.03, 80.03, 80.03, 80.03, 80.03, 80.03, 80.03]
 
Device is cuda:1
Using cloob model. Embedding path: embedding_path_cloob_rn50_yfcc_epoch_28
Evaluating zero-shot accuracy for the class: ['Cardiomegaly', 'no Cardiomegaly']
Evaluating one-shot threshold accuracy for the image dataset
Evaluating threshold: 0.4
Evaluating threshold: 0.5
Evaluating threshold: 0.6
Evaluating threshold: 0.7
Evaluating threshold: 0.8
Evaluating threshold: 0.9
Evaluating threshold: 0.95
Evaluating thres

In [12]:
df_accuracy_one = df_accuracy_one.groupby(['model', 'threshold']).mean()
df_accuracy_one = df_accuracy_one.reset_index()
df_accuracy_one

,model,threshold,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,clip_rn50_yfcc_epoch_28.pt,0.400,69.57,75.65,26.69,12.09,96.23,87.10,4.60,28.52,22.94,0.87,90.59,58.57,29.49
1,clip_rn50_yfcc_epoch_28.pt,0.500,69.57,75.65,26.69,12.09,96.23,87.10,4.60,28.52,22.94,0.87,90.59,58.57,29.49
2,clip_rn50_yfcc_epoch_28.pt,0.600,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
3,clip_rn50_yfcc_epoch_28.pt,0.700,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
4,clip_rn50_yfcc_epoch_28.pt,0.800,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
5,clip_rn50_yfcc_epoch_28.pt,0.900,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
6,clip_rn50_yfcc_epoch_28.pt,0.950,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
7,clip_rn50_yfcc_epoch_28.pt,0.990,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
8,clip_rn50_yfcc_epoch_28.pt,0.999,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53
9,cloob_rn50_yfcc_epoch_28.pt,0.400,65.77,25.67,85.86,87.72,6.85,98.40,97.14,77.44,39.65,57.40,93.02,10.14,41.89


In [13]:
# overall accuracy excluding the threshold column and the model column
df_accuracy_one['overall_accuracy'] = df_accuracy_one.iloc[:, 2:].mean(axis=1)
df_accuracy_one

,model,threshold,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,overall_accuracy
0,clip_rn50_yfcc_epoch_28.pt,0.400,69.57,75.65,26.69,12.09,96.23,87.10,4.60,28.52,22.94,0.87,90.59,58.57,29.49,46.377692
1,clip_rn50_yfcc_epoch_28.pt,0.500,69.57,75.65,26.69,12.09,96.23,87.10,4.60,28.52,22.94,0.87,90.59,58.57,29.49,46.377692
2,clip_rn50_yfcc_epoch_28.pt,0.600,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
3,clip_rn50_yfcc_epoch_28.pt,0.700,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
4,clip_rn50_yfcc_epoch_28.pt,0.800,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
5,clip_rn50_yfcc_epoch_28.pt,0.900,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
6,clip_rn50_yfcc_epoch_28.pt,0.950,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
7,clip_rn50_yfcc_epoch_28.pt,0.990,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
8,clip_rn50_yfcc_epoch_28.pt,0.999,80.03,83.20,95.22,87.91,96.81,98.42,97.30,77.46,77.06,99.13,93.20,95.74,70.53,88.616154
9,cloob_rn50_yfcc_epoch_28.pt,0.400,65.77,25.67,85.86,87.72,6.85,98.40,97.14,77.44,39.65,57.40,93.02,10.14,41.89,60.534615
